In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('iris.csv')
df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [4]:
x=df.drop(['species'],axis=1)
y=df['species']

# Preprocessing

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,LabelEncoder


In [6]:
num_pipe=Pipeline(steps=[('impute',SimpleImputer(strategy='median')),('Scaler',StandardScaler())])

In [7]:
import pickle

In [8]:
with open('pipe_raw.pkl','wb') as file:
    pickle.dump(num_pipe,file)

In [9]:
x1=pd.DataFrame(num_pipe.fit_transform(x),columns=num_pipe.get_feature_names_out())
x1

,sepal_length,sepal_width,petal_length,petal_width
0,-0.900681,1.019004,-1.340227,-1.315444
1,-1.143017,-0.131979,-1.340227,-1.315444
2,-1.385353,0.328414,-1.397064,-1.315444
3,-1.506521,0.098217,-1.283389,-1.315444
4,-1.021849,1.249201,-1.340227,-1.315444
...,...,...,...,...
145,1.038005,-0.131979,0.819596,1.448832
146,0.553333,-1.282963,0.705921,0.922303
147,0.795669,-0.131979,0.819596,1.053935
148,0.432165,0.788808,0.933271,1.448832


In [10]:
le_raw=LabelEncoder()

In [25]:
with open('le_raw.pkl','wb') as file:
    pickle.dump(le_raw,file)

In [11]:
y1=le_raw.fit_transform(y)
y1

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [12]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x1,y1,test_size=0.2,random_state=23)

## Model

In [13]:
from sklearn.neighbors import KNeighborsClassifier

In [14]:
kn=KNeighborsClassifier()

In [15]:
from sklearn.model_selection import GridSearchCV

In [16]:
grid={'n_neighbors':[2,3,4,5,6,7,8]}

In [17]:
gs=GridSearchCV(kn,grid)
gs.fit(x_train,y_train)

GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [2, 3, 4, 5, 6, 7, 8]})

In [18]:
best_knn=gs.best_estimator_
best_knn

KNeighborsClassifier()

In [19]:
gs.best_params_

{'n_neighbors': 5}

In [20]:
gs.best_estimator_

KNeighborsClassifier()

In [21]:
with open('model_raw.pkl','wb') as file:
    pickle.dump(best_knn,file)

In [23]:
y_pred=best_knn.predict(x_test)
y_pred

array([2, 2, 1, 0, 2, 1, 0, 2, 0, 1, 1, 0, 2, 0, 0, 1, 1, 1, 2, 0, 2, 0,
       0, 0, 2, 0, 0, 2, 1, 1])

In [24]:
label=le_raw.inverse_transform(y_pred)
label

array(['virginica', 'virginica', 'versicolor', 'setosa', 'virginica',
       'versicolor', 'setosa', 'virginica', 'setosa', 'versicolor',
       'versicolor', 'setosa', 'virginica', 'setosa', 'setosa',
       'versicolor', 'versicolor', 'versicolor', 'virginica', 'setosa',
       'virginica', 'setosa', 'setosa', 'setosa', 'virginica', 'setosa',
       'setosa', 'virginica', 'versicolor', 'versicolor'], dtype=object)

In [30]:
def predict_point(le,model,pipe):
    sepal_lenght=float(input('sepal_length: '))
    sepal_width=float(input('sepal_width: '))
    petal_length=float(input('petal_length: '))
    petal_width = float(input('Petal_width '))

    x_new=pd.DataFrame([sepal_lenght,sepal_width,petal_length,petal_width]).T
    x_new.columns=['sepal_length','sepal_width','petal_length','petal_width']

    print('Dataframe Before Preprocessing:\n',x_new)

    x_pre=pd.DataFrame(pipe.transform(x_new),columns=x_new.columns)
    print('DataFrame After Preprocessing: \n',x_pre)

    pred=model.predict(x_pre)

    label=le.inverse_transform(pred)[0]
    prob=model.predict_proba(x_pre).max()

    print(f'Class is {label} having probability {prob}')

    return label,prob

In [31]:
predict_point(le_raw,best_knn,num_pipe)

Dataframe Before Preprocessing:
    sepal_length  sepal_width  petal_length  petal_width
0           1.0          2.0           1.0          1.0
DataFrame After Preprocessing: 
    sepal_length  sepal_width  petal_length  petal_width
0     -5.868564    -2.433947     -1.567576    -0.262387
Class is setosa having probability 0.8


('setosa', 0.8)